---
title: "{marginaleffects} playground"
date: 2025-04-08
author: "Andrew Heiss"
toc: true
---

In [ ]:
import polars as pl
import numpy as np
import statsmodels.formula.api as smf
from marginaleffects import *

happiness = pl.read_csv("data/happiness.csv", null_values = "NA")
happiness = happiness.with_columns(
    happiness["latin_america"].cast(pl.Int32),
    happiness["happiness_binary"].cast(pl.Int32),
    happiness["region"].cast(pl.Categorical),
    happiness["income"].cast(pl.Categorical)
)

# Sliders and switches

In [ ]:
#| warning: false
model_slider = smf.ols(
  "happiness_score ~ life_expectancy", 
  data = happiness).fit()
model_slider.summary()

In [ ]:
#| warning: false
plot_predictions(model_slider, condition = "life_expectancy")

In [ ]:
#| warning: false
model_switch = smf.ols(
  "happiness_score ~ latin_america", 
  data = happiness).fit()
model_switch.summary()

In [ ]:
#| warning: false
plot_predictions(model_switch, condition = "latin_america")

In [ ]:
model_mixer = smf.ols(
  "happiness_score ~ life_expectancy + school_enrollment + C(region)", 
  data = happiness.to_pandas()).fit()
model_mixer.summary()

# Basic examples

## Happiness and life expectancy

In [ ]:
#| warning: false
model_mixer = smf.ols(
  "happiness_score ~ life_expectancy + school_enrollment + C(region)", 
  data = happiness.to_pandas()).fit()
avg_comparisons(model_mixer, variables = "life_expectancy")

## Difference between North America and South Asia

In [ ]:
#| warning: false
avg_comparisons(
  model_mixer, 
  variables = {"region": ["North America", "South Asia"]}
)

Same results from a regular regression table, but it takes a lot more work!

In [ ]:
model_mixer.summary()

Are these significantly different from each other?

In [ ]:
#| warning: false
avg_comparisons(
  model_mixer, 
  variables = {"region": ["North America", "South Asia"]},
  hypothesis=0
)

Is it significantly different from -1?

In [ ]:
#| warning: false
avg_comparisons(
  model_mixer, 
  variables = {"region": ["North America", "South Asia"]},
  hypothesis=-1
)

## Squared life expectancy

In [ ]:
model_poly = smf.ols(
  "happiness_score ~ life_expectancy + I(life_expectancy**2) + school_enrollment + latin_america", 
  data = happiness.to_pandas()).fit()
model_poly.summary()

In [ ]:
#| warning: false
avg_comparisons(
  model_poly, 
  newdata = datagrid(life_expectancy = [60, 80]), 
  by = "life_expectancy", 
  variables="life_expectancy"
)

In [ ]:
#| warning: false
plot_predictions(
  model_poly,
  condition="life_expectancy"
)

Are those two slopes significantly different from each other?

In [ ]:
#| warning: false
avg_comparisons(
  model_poly, 
  newdata = datagrid(life_expectancy = [60, 80]), 
  by = "life_expectancy", 
  variables="life_expectancy",
  hypothesis = "difference ~ pairwise"
)

## Interaction terms

In [ ]:
#| warning: false
model_poly_int = smf.ols(
    "happiness_score ~ life_expectancy * latin_america + I(life_expectancy**2) * latin_america + school_enrollment",
    data=happiness.to_pandas()
).fit()
model_poly_int.summary()

In [ ]:
#| warning: false
plot_predictions(
  model_poly_int,
  condition = ["life_expectancy", "latin_america"]
)

In [ ]:
#| warning: false
avg_comparisons(
  model_poly_int,
  newdata = datagrid(life_expectancy = [60, 80], latin_america = [0, 1]), 
  by = ["life_expectancy", "latin_america"], 
  variables = "life_expectancy")

## Synthetic data

In [ ]:
#| warning: false
predictions(
  model_poly_int, 
  newdata = datagrid(life_expectancy = [60, 80], latin_america = [0, 1])
)

# Logistic regression

In [ ]:
model_logit = smf.logit(
    formula="happiness_binary ~ life_expectancy + school_enrollment + latin_america",
    data=happiness.to_pandas()
).fit()
model_logit.summary()

Ew log odds ↑

Kinda okay odds ratios ↓

In [ ]:
np.exp(model_logit.params)

In [ ]:
#| warning: false
plot_predictions(
  model_logit, 
  condition=["life_expectancy", "latin_america"]
)

Probability scale predictions

In [ ]:
#| warning: false
predictions(
  model_logit, 
  newdata = datagrid(life_expectancy = [60, 80], latin_america = [0, 1])
)

Probability scale slopes!

In [ ]:
#| warning: false
comparisons(
  model_logit, 
  newdata = datagrid(life_expectancy = [60, 80], latin_america = [0, 1]),
  by = ["life_expectancy", "latin_america"], 
  variables = "life_expectancy"
)